# Clone content between portals

This ArcGIS Notebook app can be used to clone content -- like hosted feature layers, web maps, and web apps -- between portals/organizations. It combines this [ArcGIS Developer's documentation](https://developers.arcgis.com/python/guide/cloning-content/) on cloning items with this [iPython widgets tutorial](https://www.youtube.com/watch?v=abXuRM0QVD0) to make the ArcGIS API for Python's *clone content* functionality more accessible to non-Python users.

**You do not need to change/edit any of the code to use this ArcGIS Notebook app.** Just run each code cell using the &#9658; Run button and then enter the relevant information in the widgets after each cell. Make sure all the relevant information has been entered into a widget before running the next cell. Not all code cells return a widget, but when one does, you'll have to manually select the next code cell to run after entering the relevant information into the widget.

### Import modules

Run this code cell to import the necessary modules. No widget input is needed here.

In [ ]:
from arcgis.gis import GIS
from ipywidgets import widgets

### Widget functions

Run this code cell to define functions that will be used by widgets later. No widget input is needed here.

In [ ]:
def check_login(b):
    try:
        GIS(portal_url.value, username.value, password.value)
        login_valid.value = True
        login_valid.layout.visibility = 'visible'
    except Exception:
        login_valid.value = False
        login_valid.layout.visibility = 'visible'

### Instantiate source and target GIS objects

Run this code cell to generate a login widget to sign in to your target portal (the portal you're cloning content *to*). Click the 'Sign in' button to confirm the URL/credentials work (if they do not, a red 'x' will appear). Once you've signed in, manually select the next code cell and run it.

In [ ]:
# define widgets
portal_url = widgets.Text(description='Portal URL', value='https://www.arcgis.com', placeholder='https://www.arcgis.com')
username = widgets.Text(description='Username')
password = widgets.Password(description='Password')
login_button = widgets.Button(description='Sign in', button_style='Success')
login_valid = widgets.Valid()

# define HTML
login_html = widgets.HTML('<h3><font color=\'#42a5f5\'>Sign in to your target portal</font></h3>')

# organize layout/display
login_button.style.button_color = '#42a5f5'
login_button.layout.width = 'auto'
login_valid.layout.visibility = 'hidden'
login_row = widgets.HBox([login_button, login_valid])
login = widgets.VBox([login_html, portal_url, username, password, login_row])

# set button action
login_button.on_click(check_login)

login

In [ ]:
source = GIS('home')
target = GIS(portal_url.value, username.value, password.value)

### Search for items to clone

Run this code cell to generate a search box widget. Once it generates, enter your search term(s) in it and then manually select the next cell and run it. Note: the search term is automatically saved by the widget, all you have to do is enter your term into the search box.

In [ ]:
# define widgets
search_input = widgets.Text(description='Search term', placeholder='e.g. Washington Census Blocks')

# define HTML
search_html = widgets.HTML('<h3><font color=\'#42a5f5\'>Search for content to clone</font></h3>')

# organize layout/display
search_layout = widgets.VBox([search_html, search_input])

search_layout

In [ ]:
search_results = source.content.search(f'{search_input.value} AND owner: {source.users.me.username}')

# define widgets
search_select = widgets.SelectMultiple(
    options=[f'{item.title} ({item.type}) ({item.id})' for item in search_results],
    value=[],
    rows=len(search_results),
    description='Results',
    disabled=False
)

# define HTML
results_html = widgets.HTML('''<h3><font color=\'#42a5f5\'>Select content to clone</font></h3>
                            <p>Select multiple items using shift or ctrl/command. Nothing is selected by default, so make sure to make at leaxt one selection.</p>''')

# organize layout/display
search_select.layout.width = '80%'
results_layout = widgets.VBox([results_html, search_select])

results_layout

### Clone selected content

Run this code cell to clone the content selected in the previous widget. A list of the successfully cloned items in the target portal should appear below the cell. If you recieve an error, referer to the ArcGIS API for Python's clone conent/.clone_items() documentation \[[1](https://developers.arcgis.com/python/guide/cloning-content/)\] \[[2](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?highlight=clone#arcgis.gis.ContentManager.clone_items)\].

In [ ]:
content_to_clone = []

for item in search_select.value:
    content_to_clone.append(source.content.get(item.split()[-1][1:-1]))
    
target.content.clone_items(items=content_to_clone)

### Sources

+ [ipython widgets tutorial](https://www.youtube.com/watch?v=abXuRM0QVD0)
+ [ipython widgets doc](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
+ [clone_items() doc](https://developers.arcgis.com/python/guide/cloning-content/)